## Leitura do Dataset Movies que contém títulos e generos de 27.278 filmes 

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
import matplotlib.pyplot as plt
import warnings

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report
from unicodedata import normalize

nltk.download('wordnet')
warnings.filterwarnings('ignore')

In [ ]:
import numpy as nu

In [ ]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def trata_caracteres(txt):
    txt=[normalize('NFKD',x).encode('ASCII','ignore').decode('ASCII') for x in txt]
    txt=[re.sub('[^a-z|\ ]','',str.lower(x)) for x in txt]
    return txt

In [ ]:
stop_words = set(stopwords.words('english'))
def remove_stop_words(txt):
    texto = CountVectorizer(ngram_range=(1,1),stop_words=stop_words)
    texto.fit(txt)
    return texto

In [ ]:
def tratar_texto(txt,metodo=''):
    txt = trata_caracteres(txt)
    try:
        texto = remove_stop_words(txt)
        lista_txt = texto.get_feature_names()
    except:
        lista_txt = txt
    if metodo=='lemma':
        lem = WordNetLemmatizer()
        for part_of_speech in ['a', 's', 'r', 'n', 'v']:
            tms = [lem.lemmatize(a,part_of_speech) for a in lista_txt]
    if metodo == 'stem':
        ps = PorterStemmer()
        tms = [ps.stem(a) for a in lista_txt]
    if metodo == '':
        tms = txt
    return " ".join(tms)

In [ ]:
data = pd.read_csv('movies.csv',index_col=0)

In [ ]:
data['title'] = data.title.apply(lambda x: tratar_texto([x],'stem'))

In [ ]:
data

In [ ]:
data.describe()

In [ ]:
data = data.dropna()
data.describe()

No dataset não existem títulos sem classificação de gênero e vice-versa.

In [ ]:
qtd = data.groupby(['title']).size().reset_index(name='counts')

In [ ]:
print("Temos",len(qtd[qtd['counts']==2]),"títulos com mesmo nome e classificação de gênero")

In [ ]:
qtd.sort_values(by='counts',ascending=False)


In [ ]:
for dado in [d for d in data['title'] if d in list(qtd[qtd['counts']==2].title)]:
    print(data[data['title']==dado])

Eliminamos do dataset as duplicações de títulos que se repetiram; mesmo título e mesma classificação de gênero.

Encontramos ainda 10 títulos que possuem classificação diferentes. Em alguns casos em uma das classificações o gênero já está contido no conjunto, mas em outros não.

Computar o DTM

In [ ]:
def remove_ano(txt):
    txt=re.sub(r'(\([^a-z]*\))','',str.lower(txt))
    return txt

In [ ]:
data.update(data.title.apply(remove_ano))

In [ ]:
data =data.drop_duplicates()
data.describe()

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def texto_tokenize(txt):
    return [word_tokenize(t) for t in txt]
wt = texto_tokenize(data.title)

In [ ]:
y = [x for x in[a for a in wt]]

In [ ]:
query = ['toy story', 'the lion king','alladin','beauty and the best','cinderella','little mermaid','hercules']

In [ ]:
query

In [ ]:
vect = CountVectorizer(ngram_range=(1,1))
H = vect.fit(data.title)
o = vect.transform(data.title)

In [ ]:
Dc = vect.transform([tratar_texto([frase],'stem') for frase in query])

In [ ]:
DTM = pd.DataFrame(o.A, columns=vect.get_feature_names()).T

In [ ]:
x = DTM
m = x.mean(axis=0)
c = 2
B = c*(1-m)
a = c*m


In [ ]:
a_til = a + x.sum()

In [ ]:
N = len(Dc.sum(axis=1))

In [ ]:
B_til = B + N - x.sum()

In [ ]:
q = nu.log(a_til) - nu.log(a) - nu.log(B_til) + nu.log(B)

In [ ]:
nc = nu.log(a + B).sum() - nu.log(a + B + N) + nu.log(B_til) - nu.log(B)

In [ ]:
s =nc + x.multiply(q)

In [ ]:
s.T